# Reconnaissance de sentiment sur des critiques de films

## Paramètres du notebook

In [46]:
corpus_language = "english"
folder_path = "./datasets/movies/"
corpus_base_path = folder_path + "movies1000/"

file_name_test = folder_path + "testSentiment.txt"
file_name_output = corpus_base_path + "testSentiment.pred.txt"

results_save_path = "../Reports/report_project/results/movies/"

files_to_ignore = [".DS_Store", "README"]

In [2]:
training = {"small": 30, "medium": 200, "large": 2000}
study_config = "medium"
final_config = "large"

# minimum number of uppercase characters in a word to be considered as uppercase
# (e.g. "upper" + "HELLO" = "upperhello") if uppercase_markers is True
# during preprocessing
min_upper_char = 3

In [3]:
sklearn_working_memory = 24 * 1024

## Imports

In [4]:
# Imports bibliothèques Python
import codecs
import re
import string
import os

In [5]:
# Imports bibliothèques externes
import numpy as np
import nltk
import sklearn

sklearn.set_config(working_memory=sklearn_working_memory)

In [6]:
# Téléchargement des ressources nécessaires à NLTK
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /Users/mathis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mathis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mathis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# Paramètres matplotlib
import matplotlib

matplotlib.use("pgf")
import matplotlib.pyplot as plt

matplotlib.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "text.usetex": True,
        "pgf.rcfonts": False,
    }
)

# 1) Chargement des données

In [8]:
def load_movies(path):  # 1 classe par répertoire
    texts_list, labels_list = [], []
    directories = [d for d in os.listdir(path) if d not in files_to_ignore]
    for classe in directories:
        files = [file for file in os.listdir(path + classe) if file not in files_to_ignore]
        for file in files:
            file_path = path + classe + "/" + file
            opened_file = codecs.open(file_path, "r", "utf-8")
            texts_list.append(opened_file.read())
            if classe == "pos":
                labels_list.append("Positive")
            else:
                labels_list.append("Negative")
    return texts_list, labels_list

In [9]:
texts, labels = load_movies(corpus_base_path)

## Données équilibrées

In [10]:
n = len(labels)
print(n, "exemples")
for sentiment in ["Positive", "Negative"]:
    count = labels.count(sentiment)
    percentage = count / n * 100
    print(f"Exemples {sentiment} : {count} ({percentage:.2f}%)")

2000 exemples
Exemples Positive : 1000 (50.00%)
Exemples Negative : 1000 (50.00%)


# 2) Traitement des données

## a) Prétraitement

In [11]:
# A pattern to match continuous uppercase substrings,
# including accented uppercase characters
# Only matches substrings of min_upper_char or more characters
upper = r"\b(?:[A-ZÀ-ÖØ-öø-ÿĀ-ſƀ-ɏḀ-ỿꜢ-ꞇﬀ-ﬆＡ-Ｚ]{{{},}})\b"
upper_case_pattern = upper.format(min_upper_char)
upper_case_pattern_compiled = re.compile(pattern=upper_case_pattern)
upper_replacement = lambda m: "upper" + m.group(0).lower()


def custom_preprocessor(
    text, remove_punctuation=True, remove_numbers=True, uppercase_markers=False
):
    if uppercase_markers:
        text = re.sub(
            pattern=upper_case_pattern_compiled, repl=upper_replacement, string=text
        )

    if remove_punctuation:
        text = text.translate(str.maketrans("", "", string.punctuation))

    if remove_numbers:
        text = text.translate(str.maketrans("", "", string.digits))

    return text

## b) Tokenization

In [12]:
def custom_tokenizer(text, stemmer=None, lemmatizer=None):
    from nltk.tokenize import word_tokenize

    tokens = word_tokenize(text, language=corpus_language)

    assert (
        stemmer is None or lemmatizer is None
    ), "Only one of stemmer or lemmatizer can be used."

    if stemmer is not None:
        tokens = [stemmer.stem(token) for token in tokens]

    if lemmatizer is not None:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

## c) Transformation occurrences -> TF-IDF

Sera systématiquement appliqué si un modèle utilisant LSA comme réduction de dimension est utilisé

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

param_tfidftransformer = {
    "transformer": [TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)],
    "transformer__norm": ["l1", "l2", None],
}

param_grid_tfidf_transformer = [
    {
        "transformer": [None],
    },
    param_tfidftransformer,
]

## d) Réduction de la dimension

Ne sera pas employé sur le classifieur Naive Bayes

In [14]:
from scipy.stats import uniform, norm, randint

In [15]:
from sklearn.decomposition import TruncatedSVD

param_grid_dim_reduction = [
    {
        "dim_reduction": [None],
    },
    {
        "dim_reduction": [TruncatedSVD()],
        "dim_reduction__n_components": randint(10, 30),
        "dim_reduction__algorithm": ["randomized", "arpack"],
    },
]

# 3) Modèles étudiés

## a) Classifieur Naive Bayes

In [16]:
from sklearn.naive_bayes import MultinomialNB

multinomial_classifier = MultinomialNB(fit_prior=False)
nb_regul_distrib = uniform(loc=0.8, scale=0.2)
param_grid_nb = {
    "classifier": [multinomial_classifier],
    "classifier__alpha": nb_regul_distrib,
}

## b) Classifieur SVM

In [17]:
from sklearn.svm import LinearSVC

linear_svc_classifier = LinearSVC(dual=False)
svm_regul_distrib = norm(loc=1.0, scale=0.2)
param_grid_svm = {
    "classifier": [linear_svc_classifier],
    "classifier__C": svm_regul_distrib,
    "classifier__penalty": ["l1", "l2"],
}

## c) Classifieur régression logistique

In [18]:
from sklearn.linear_model import LogisticRegression

log_reg_classifier = LogisticRegression(solver="liblinear")
log_reg_regul_distrib = norm(loc=1.0, scale=0.2)
param_grid_log_reg = {
    "classifier": [log_reg_classifier],
    "classifier__C": log_reg_regul_distrib,
    "classifier__penalty": ["l1", "l2"],
}

## Grille de paramètres des classifieurs

In [19]:
param_grid_classifier = [
    param_grid_nb,
    param_grid_svm,
    param_grid_log_reg,
]

# 4) Metrique de selection de modèles

In [20]:
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

# from sklearn.metrics import accuracy_score

# accuracy score
# scorer = make_scorer(accuracy_score)

In [21]:
scorer = make_scorer(f1_score, average="binary", pos_label="Positive")

# 5) Pipeline

## a) Assemblage du pipeline

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    steps=
    [
        ("vectorizer", CountVectorizer(token_pattern=None)),
        ("transformer", None),
        ("dim_reduction", None),
        ("scaler", None),
        ("classifier", None),
    ]
)

## b) Grille de recherche

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
from nltk.stem import SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer

In [26]:
param_grid = [
    {
        "vectorizer__binary": [True, False],
        "vectorizer__lowercase": [True, False],
        "vectorizer__max_features": [None, 2000, 5000, 10000],
        "vectorizer__strip_accents": ["ascii", "unicode", None],
        "vectorizer__preprocessor": [
            lambda text: custom_preprocessor(
                text,
                remove_punctuation=remove_punct,
                remove_numbers=remove_nums,
                uppercase_markers=uppercase_marks,
            )
            for remove_punct in [True, False]
            for remove_nums in [True, False]
            for uppercase_marks in [True, False]
        ],
        "vectorizer__tokenizer": [
            lambda text: custom_tokenizer(text, stemmer=stem, lemmatizer=lem)
            for stem, lem in [
                (None, None),
                (PorterStemmer(), None),
                (SnowballStemmer(language=corpus_language), None),
                (LancasterStemmer(), None),
                (None, WordNetLemmatizer()),
            ]
        ],
        # min_df and max_df instead of stopwords list
        "vectorizer__min_df": uniform(loc=0.0, scale=0.02),
        "vectorizer__max_df": uniform(loc=0.5, scale=0.45),
        "vectorizer__ngram_range": [(1, 1), (1, 2), (2, 2)],
        **dim_reduction_params,
        **(
            # tfidf transformer is recommended when dimensionality reduction is used
            transformer_params
            if dim_reduction_params["dim_reduction"] is None
            else param_tfidftransformer
        ),
        # scaler useless with multinomial naive bayes classifier
        # mean = False to avoid transforming the sparse matrix into a dense one
        "scaler": [
            None
            if isinstance(classifier_params["classifier"][0], MultinomialNB)
            else StandardScaler(with_mean=False)
        ],
        **classifier_params,
    }
    for classifier_params in param_grid_classifier
    for transformer_params in param_grid_tfidf_transformer
    for dim_reduction_params in (
        [param_grid_dim_reduction[0]]
        if isinstance(classifier_params["classifier"][0], MultinomialNB)
        else param_grid_dim_reduction[1:]
    )
]

# 6) Recherche du meilleur modèle

## Split train/test

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42, shuffle=True
)

## K-Fold

In [28]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=4, shuffle=True, random_state=42)

## Recherche : Halving Random Search

In [29]:
# noinspection PyUnresolvedReferences
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

search = HalvingRandomSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    scoring=scorer,
    cv=cv,
    factor=1.5,
    n_candidates=training[study_config],
    min_resources=800,
    max_resources="auto",
    aggressive_elimination=True,
    n_jobs=-1,
    refit=True,
    verbose=1,
)

fitted_search = search.fit(X_train, y_train)

n_iterations: 14
n_required_iterations: 14
n_possible_iterations: 2
min_resources_: 800
max_resources_: 1600
aggressive_elimination: True
factor: 1.5
----------
iter: 0
n_candidates: 200
n_resources: 800
Fitting 4 folds for each of 200 candidates, totalling 800 fits


/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWa

----------
iter: 1
n_candidates: 134
n_resources: 800
Fitting 4 folds for each of 134 candidates, totalling 536 fits


/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

----------
iter: 2
n_candidates: 90
n_resources: 800
Fitting 4 folds for each of 90 candidates, totalling 360 fits
----------
iter: 3
n_candidates: 60
n_resources: 800
Fitting 4 folds for each of 60 candidates, totalling 240 fits


/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


----------
iter: 4
n_candidates: 40
n_resources: 800
Fitting 4 folds for each of 40 candidates, totalling 160 fits
----------
iter: 5
n_candidates: 27
n_resources: 800
Fitting 4 folds for each of 27 candidates, totalling 108 fits
----------
iter: 6
n_candidates: 18
n_resources: 800
Fitting 4 folds for each of 18 candidates, totalling 72 fits
----------
iter: 7
n_candidates: 12
n_resources: 800
Fitting 4 folds for each of 12 candidates, totalling 48 fits
----------
iter: 8
n_candidates: 8
n_resources: 800
Fitting 4 folds for each of 8 candidates, totalling 32 fits
----------
iter: 9
n_candidates: 6
n_resources: 800
Fitting 4 folds for each of 6 candidates, totalling 24 fits
----------
iter: 10
n_candidates: 4
n_resources: 800
Fitting 4 folds for each of 4 candidates, totalling 16 fits
----------
iter: 11
n_candidates: 3
n_resources: 800
Fitting 4 folds for each of 3 candidates, totalling 12 fits
----------
iter: 12
n_candidates: 2
n_resources: 800
Fitting 4 folds for each of 2 candidate

# 7) Comparaison des modèles

## Sauvegarde des résultats

In [30]:
import pandas as pd


def save_results(model_search, name: str):
    results = pd.concat(
        [
            pd.DataFrame(model_search.cv_results_["params"]),
            pd.DataFrame(
                model_search.cv_results_["mean_test_score"], columns=["score"]
            ),
            pd.DataFrame(
                model_search.cv_results_["std_test_score"], columns=["std_score"]
            ),
            pd.DataFrame(model_search.cv_results_["rank_test_score"], columns=["rank"]),
        ],
        axis=1,
    )
    results.sort_values("rank", axis=0, ascending=True, inplace=True, kind="quicksort")
    results.to_csv(name + ".csv")
    return results

In [31]:
study_results_file_name = "Study_HalvingRandomSearchCV_results"
study_results_save_path = results_save_path + study_results_file_name
study_results = save_results(fitted_search, study_results_save_path)

## Affichage des résultats de la cross-validation

In [32]:
# Get the cv results
cv_results = fitted_search.cv_results_

# Extract mean and std of test scores
mean_scores = fitted_search.cv_results_["mean_test_score"]
std_scores = fitted_search.cv_results_["std_test_score"]

top_indices = np.argsort(mean_scores)[::-1][: training[study_config]]
top_mean_scores = mean_scores[top_indices]
top_std_scores = std_scores[top_indices]

In [33]:
plt.figure()
x = np.arange(training[study_config]) + 1
plt.plot(x, top_mean_scores, label="Mean score")
plt.fill_between(
    x,
    top_mean_scores - top_std_scores,
    top_mean_scores + top_std_scores,
    alpha=0.2,
    label="±1 std",
)

plt.xlabel("models")
plt.ylabel("Validation score")
plt.title(
    "Mean validation scores and standard deviations of top {} models".format(
        training[study_config]
    )
)
plt.legend(loc="upper right")
model_rank_global_file_name = "model_rank_global.pgf"
model_rank_save_path = results_save_path + model_rank_global_file_name
plt.savefig(model_rank_save_path)
plt.close()

In [34]:
# Number of top models to plot
n_top_models = min(80, training[study_config] // 3)

# Classifier colors
classifier_colors = {
    "MultinomialNB": "blue",
    "LinearSVC": "green",
    "LogisticRegression": "red",
}

# Get the indices of the top n models sorted by mean_test_score
top_n_indices = np.argsort(cv_results["mean_test_score"])[-n_top_models:][::-1]

# Plot the mean validated score and standard deviation for the top n models
fig, ax = plt.subplots()

for i, index in enumerate(top_n_indices):
    mean_score = cv_results["mean_test_score"][index]
    std_score = cv_results["std_test_score"][index]
    classifier_type = cv_results["param_classifier"][index].__class__.__name__
    color = classifier_colors.get(classifier_type, "black")
    ax.errorbar(
        i + 1,
        mean_score,
        yerr=std_score,
        fmt="",
        color=color,
        label=f"{classifier_type}",
    )

handles, ax_labels = ax.get_legend_handles_labels()
by_label = dict(zip(ax_labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc="upper right")

ax.set_xlabel("models")
ax.set_ylabel("Validation score")
ax.set_title(
    "Mean validation scores and standard deviations for the top {} models".format(
        n_top_models
    )
)

model_rank_classifier_file_name = "model_rank_classifier.pgf"
model_rank_classifier_save_path = results_save_path + model_rank_classifier_file_name
plt.savefig(model_rank_classifier_save_path)

plt.close()

# 8) Meilleur pipeline

## Meilleur score

In [35]:
best_pipeline = fitted_search.best_estimator_
best_classifier = best_pipeline.named_steps["classifier"]

print("Best pipeline:", best_pipeline)
print("Best classifier:", best_classifier)

Best pipeline: Pipeline(steps=[('vectorizer',
                 CountVectorizer(binary=True, lowercase=False,
                                 max_df=0.7790294902762321, max_features=10000,
                                 min_df=0.0034795340308673883,
                                 ngram_range=(1, 2),
                                 preprocessor=<function <listcomp>.<listcomp>.<lambda> at 0x167af9440>,
                                 token_pattern=None,
                                 tokenizer=<function <listcomp>.<listcomp>.<lambda> at 0x167af9760>)),
                ('transformer', TfidfTransformer(norm=None, sublinear_tf=True)),
                ('dim_reduction', None), ('scaler', None),
                ('classifier',
                 MultinomialNB(alpha=0.9124879001831941, fit_prior=False))])
Best classifier: MultinomialNB(alpha=0.9124879001831941, fit_prior=False)


## Evaluation finale du meilleur modèle

In [36]:
y_pred = best_pipeline.predict(X_test)

In [37]:
from sklearn.metrics import classification_report

print("Classification report:")
print(classification_report(y_test, y_pred, digits=3))

Classification report:
              precision    recall  f1-score   support

    Negative      0.848     0.840     0.844       200
    Positive      0.842     0.850     0.846       200

    accuracy                          0.845       400
   macro avg      0.845     0.845     0.845       400
weighted avg      0.845     0.845     0.845       400



## Matrice de confusion

In [38]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred,
    normalize="all",
    cmap="winter",
    values_format=".3f",
    text_kw={"fontsize": 18, "color": "red", "fontweight": "bold"},
)

## Courbe ROC / Precision-Recall

In [39]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_estimator(
    best_pipeline, X_test, y_test, name="ROC curve", drop_intermediate=True
)

plt.savefig(results_save_path + "roc_curve.pgf")
plt.close()

In [40]:
from sklearn.metrics import PrecisionRecallDisplay

PrecisionRecallDisplay.from_estimator(
    best_pipeline, X_test, y_test, name="Precision-Recall curve"
)

plt.savefig(results_save_path + "precision_recall_curve.pgf")
plt.close()

# 9) Application sur les données sans labels

## Nouvelle recherche avec cross-validation sur l'ensemble des données
Le modèle final ne sera pas évalué sur des données de test.
L'évaluation se fera lors de la soumission des prédictions sur le serveur de test

In [41]:
final_search = fitted_search.set_params(
    **{"n_candidates": training[final_config],
       "factor": 1.5,
       "refit": True
       }
)

In [42]:
final_fitted_search = final_search.fit(texts, labels)

n_iterations: 19
n_required_iterations: 19
n_possible_iterations: 2
min_resources_: 800
max_resources_: 2000
aggressive_elimination: True
factor: 1.5
----------
iter: 0
n_candidates: 2000
n_resources: 800
Fitting 4 folds for each of 2000 candidates, totalling 8000 fits


/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWa

----------
iter: 1
n_candidates: 1334
n_resources: 800
Fitting 4 folds for each of 1334 candidates, totalling 5336 fits


/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

----------
iter: 2
n_candidates: 890
n_resources: 800
Fitting 4 folds for each of 890 candidates, totalling 3560 fits


...

----------
iter: 3
n_candidates: 594
n_resources: 800
Fitting 4 folds for each of 594 candidates, totalling 2376 fits
----------
iter: 4
n_candidates: 396
n_resources: 800
Fitting 4 folds for each of 396 candidates, totalling 1584 fits
----------
iter: 5
n_candidates: 264
n_resources: 800
Fitting 4 folds for each of 264 candidates, totalling 1056 fits


/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


----------
iter: 6
n_candidates: 176
n_resources: 800
Fitting 4 folds for each of 176 candidates, totalling 704 fits


/Users/mathis/.config/.pyenv/versions/PLDAC/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


----------
iter: 7
n_candidates: 118
n_resources: 800
Fitting 4 folds for each of 118 candidates, totalling 472 fits
----------
iter: 8
n_candidates: 79
n_resources: 800
Fitting 4 folds for each of 79 candidates, totalling 316 fits
----------
iter: 9
n_candidates: 53
n_resources: 800
Fitting 4 folds for each of 53 candidates, totalling 212 fits
----------
iter: 10
n_candidates: 36
n_resources: 800
Fitting 4 folds for each of 36 candidates, totalling 144 fits
----------
iter: 11
n_candidates: 24
n_resources: 800
Fitting 4 folds for each of 24 candidates, totalling 96 fits
----------
iter: 12
n_candidates: 16
n_resources: 800
Fitting 4 folds for each of 16 candidates, totalling 64 fits
----------
iter: 13
n_candidates: 11
n_resources: 800
Fitting 4 folds for each of 11 candidates, totalling 44 fits
----------
iter: 14
n_candidates: 8
n_resources: 800
Fitting 4 folds for each of 8 candidates, totalling 32 fits
----------
iter: 15
n_candidates: 6
n_resources: 800
Fitting 4 folds for each o

In [43]:
final_results_file_name = "Final_HalvingRandomSearchCV_results"
final_results_save_path = results_save_path + study_results_file_name
final_results = save_results(final_fitted_search, final_results_save_path)

In [44]:
final_model = final_fitted_search.best_estimator_

## Prédiction

In [56]:
opened_file = codecs.open(file_name_test, "r", "utf-8")
texts = opened_file.read().split("\n")[:-1]
y_pred_test = final_model.predict(texts)

# Sauvegarde des prédictions
with open(file_name_output, "w") as f:
    for sentiment in y_pred_test:
        f.write(sentiment[0] + "\n")
